In [1]:
# Load embeddings
from pickle import load
import numpy as np
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

embeds = load(open("jepa_encodings.pkl", "rb"))

# Break into categories
clothing_embed = {k:v for k, v in embeds.items() if k.endswith("c.png")}
clothing_labels = [k.split("-")[0].split("_") for k in clothing_embed.keys()]
clothing_labels = [[lab[0], lab[1], lab[2][:-1]] if lab[2].endswith("s") else lab for lab in clothing_labels]
clothing_embed = np.array(list(clothing_embed.values()))

full_body_embed = {k:v for k, v in embeds.items() if k.endswith("fb.png")}
full_body_labels = [k.split("-")[0].split("_") for k in full_body_embed.keys()]
full_body_labels = [[lab[0], lab[1], lab[2][:-1]] if lab[2].endswith("s") else lab for lab in full_body_labels]
full_body_embed = np.array(list(full_body_embed.values()))

In [2]:
# Preprocess embedings
clothing_genders = np.zeros((clothing_embed.shape[0], 2))
clothing_genders[np.array([{"mens": 0, "womens": 1}[lab[1]] for lab in clothing_labels])] = 1

clothing_type = np.zeros((clothing_embed.shape[0], 6))
clothing_type[np.array([
    {"shirt": 0, "top": 1, "sweater": 2, "pant": 3, "skirt": 4, "short": 5}[lab[2]]
    for lab in clothing_labels
])] = 1

In [38]:
# Train model
import tensorflow as tf

class triplet_model(tf.keras.Model):
    def __init__(self, margin=0, layers=None, trainable=True,):
        super().__init__(trainable=trainable)
        self.clothing_model = tf.keras.Sequential(layers)
        self.outfit_model = tf.keras.Sequential(layers)
        self.margin = margin

    # https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit#supporting_sample_weight_class_weight
    # https://en.wikipedia.org/wiki/Triplet_loss
    def train_step(self, data):
        x, x_positive, x_negative = data

        with tf.GradientTape() as tape:
            y_anchor = self.clothing_model(x)
            y_positive = self.outfit_model(x_positive)
            y_negative = self.outfit_model(x_negative)
            dist_positive = tf.math.sqrt(tf.math.reduce_sum(tf.math.square(y_anchor - y_positive), axis=-1))
            dist_negative = tf.math.sqrt(tf.math.reduce_sum(tf.math.square(y_anchor - y_negative), axis=-1))
            loss = tf.math.maximum(dist_positive - dist_negative + self.margin, 0)
            loss = tf.math.reduce_sum(loss)

        # Compute gradients
        trainable_vars = self.clothing_model.trainable_variables + self.outfit_model.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics.
        # Metrics are configured in `compile()`.
        for metric in self.metrics:
            if metric.name == "loss":
                metric.update_state(loss)
            else:
                metric.update_state(tf.ones((tf.shape(x)[0], 1)), tf.expand_dims(tf.cast(dist_positive < dist_negative, float), axis=-1))

        # Return a dict mapping metric names to current value.
        # Note that it will include the loss (tracked in self.metrics).
        return {m.name: m.result() for m in self.metrics}

    def test_step(self, data):
        x, x_positive, x_negative = data

        y_anchor = self.clothing_model(x)
        y_positive = self.outfit_model(x_positive)
        y_negative = self.outfit_model(x_negative)
        
        dist_positive = tf.math.sqrt(tf.math.reduce_sum(tf.math.square(y_anchor - y_positive), axis=-1))
        dist_negative = tf.math.sqrt(tf.math.reduce_sum(tf.math.square(y_anchor - y_negative), axis=-1))
        loss = tf.math.maximum(dist_positive - dist_negative + self.margin, 0)
        loss = tf.math.reduce_sum(loss)

        # Update the metrics.
        # Metrics are configured in `compile()`.
        for metric in self.metrics:
            if metric.name == "loss":
                metric.update_state(loss)
            else:
                metric.update_state(tf.ones((tf.shape(x)[0], 1)), tf.expand_dims(tf.cast(dist_positive < dist_negative, float), axis=-1))

        # Return a dict mapping metric names to current value.
        # Note that it will include the loss (tracked in self.metrics).
        return {m.name: m.result() for m in self.metrics}


def triplet_sample_generator(validation=False):
    clothing = np.concatenate([clothing_embed, clothing_genders, clothing_type], axis=-1)
    outfit = np.concatenate([full_body_embed, clothing_genders, clothing_type], axis=-1)
    
    if validation:
        inds = np.arange(clothing_embed.shape[0])[np.arange(clothing_embed.shape[0]) % 10 == 0]
    else:
        inds = np.arange(clothing_embed.shape[0])[np.arange(clothing_embed.shape[0]) % 10 != 0]

    for i in inds:
        neg_inds = inds[inds != i]
        np.random.shuffle(neg_inds)
        yield (clothing[i], outfit[i], outfit[neg_inds[0]])

ds_train = tf.data.Dataset.from_generator(triplet_sample_generator, output_types=(float, float, float))
ds_valid = tf.data.Dataset.from_generator(lambda: triplet_sample_generator(validation=True), output_types=(float, float, float))

model = triplet_model(
    layers = [
    tf.keras.layers.Input((clothing_embed.shape[1] + 2 + 6,)),
    tf.keras.layers.Dense(clothing_embed.shape[1] * 8, activation="elu"),
    tf.keras.layers.Dense(clothing_embed.shape[1] * 8, activation="elu"),
    tf.keras.layers.Dense(clothing_embed.shape[1] * 8, activation="elu"),
    tf.keras.layers.Dense(32)],
    margin = 0.5
)

model.compile(
    optimizer="adam",
    run_eagerly=True,
    metrics=["binary_crossentropy", "binary_accuracy"]
)

model.fit(
    ds_train.batch(32),
    epochs=100,
    validation_data=ds_valid.batch(32),
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_binary_accuracy", patience=25, restore_best_weights=True)],
)   

Epoch 1/100
     18/Unknown 21s 1s/step - binary_accuracy: 0.4591 - binary_crossentropy: 8.7186 - loss: 4350.1743

2024-04-30 14:25:33.369568: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-30 14:25:33.614866: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


18/18 ━━━━━━━━━━━━━━━━━━━━ 22s 1s/step - binary_accuracy: 0.4605 - binary_crossentropy: 8.6957 - loss: 4288.2944 - val_binary_accuracy: 0.4531 - val_binary_crossentropy: 8.8146 - val_loss: 477.6611
Epoch 2/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - binary_accuracy: 0.4451 - binary_crossentropy: 8.9440 - loss: 428.3204

2024-04-30 14:25:54.700145: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-30 14:25:54.947701: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


18/18 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - binary_accuracy: 0.4462 - binary_crossentropy: 8.9254 - loss: 426.8189 - val_binary_accuracy: 0.5156 - val_binary_crossentropy: 7.8072 - val_loss: 225.2318
Epoch 3/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - binary_accuracy: 0.4979 - binary_crossentropy: 8.0924 - loss: 145.5914

...

2024-04-30 14:44:07.789742: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - binary_accuracy: 0.6655 - binary_crossentropy: 5.3920 - loss: 12.8527

2024-04-30 14:44:28.221148: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


18/18 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - binary_accuracy: 0.6647 - binary_crossentropy: 5.4043 - loss: 12.8667 - val_binary_accuracy: 0.4219 - val_binary_crossentropy: 9.3183 - val_loss: 20.9500


2024-04-30 14:44:28.461670: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [41]:
# Test model
converted_clothing_embeds = model.clothing_model(np.concatenate([clothing_embed, clothing_genders, clothing_type], axis=-1))
converted_clothing_embeds = tf.expand_dims(converted_clothing_embeds, axis=1)
print(converted_clothing_embeds)

converted_outfit_embeds = model.outfit_model(np.concatenate([full_body_embed, clothing_genders, clothing_type], axis=-1))
converted_outfit_embeds = tf.expand_dims(converted_outfit_embeds, axis=0)
print(converted_outfit_embeds)

distances = tf.math.sqrt(tf.math.reduce_sum(tf.math.square(converted_outfit_embeds - converted_clothing_embeds), axis=-1))
best_samples = tf.math.top_k(distances, k=5).indices

best_samples

tf.Tensor(
[[[-17.469297   16.743008  -25.498028  ... -18.655756   20.296185
   -10.009718 ]]

 [[-18.130577   17.98634   -26.718834  ... -18.516964   20.836916
   -11.389187 ]]

 [[-18.21362    18.31063   -26.868814  ... -18.283073   21.285402
   -11.497774 ]]

 ...

 [[-18.183187   18.895823  -26.318419  ... -18.610838   20.939468
   -10.893594 ]]

 [[-17.952415   19.119427  -25.420603  ... -18.724148   20.407167
    -9.656386 ]]

 [[-17.766253   17.9421    -25.684315  ... -18.252218   21.179535
   -10.5717945]]], shape=(640, 1, 32), dtype=float32)
tf.Tensor(
[[[-17.854269   17.261269  -25.267828  ... -17.436056   21.537453
    -9.759023 ]
  [-18.014503   17.78532   -25.35727   ... -17.184605   21.602112
    -9.9099045]
  [-17.949375   17.962849  -25.427267  ... -17.003313   21.802223
   -10.07243  ]
  ...
  [-17.978811   18.349144  -24.748835  ... -17.520416   21.412407
    -9.214986 ]
  [-17.888845   18.190914  -24.249556  ... -17.837912   21.085331
    -8.833302 ]
  [-17.902138   

<tf.Tensor: shape=(640, 5), dtype=int32, numpy=
array([[585, 218, 330, 250, 531],
       [ 11, 585, 218, 531, 330],
       [ 11, 531, 585, 218, 330],
       ...,
       [ 11,   4, 288, 266, 531],
       [  4,  11,   0,   2, 174],
       [585, 218, 330, 531, 250]], dtype=int32)>

In [ ]:
converted_clothing_embeds, tf_full_body_embed